In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras import Sequential

In [2]:
batch_size = 1024
seed = 39
sequence_length = 25
vocab_size = 10000
embedding_dim = 128

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "dataset", batch_size = batch_size, validation_split = 0.2, 
    subset = 'training', seed = seed
)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "dataset", batch_size = batch_size, validation_split = 0.2, 
    subset = 'validation', seed = seed
)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size =  AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size =  AUTOTUNE)



vectorize_layer = TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int', 
    output_sequence_length = sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

Found 230710 files belonging to 2 classes.
Using 184568 files for training.
Found 230710 files belonging to 2 classes.
Using 46142 files for validation.


In [3]:




class EmbeddingInitializer(tf.keras.initializers.Initializer):
    def __call__(self, shape=None, dtype=None, **kwargs):
        weights = tf.convert_to_tensor(np.load('w2vVectors.npy'))
        return weights

embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            embeddings_initializer = EmbeddingInitializer(),
                            mask_zero= True)




model = Sequential([
    vectorize_layer,
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="classifier_logs")

model.fit(train_ds, epochs=25,
                    validation_data=val_ds,
                    validation_steps=30, 
                    callbacks = tensorboard_callback)

Epoch 1/25
181/181 [==============================] - 38s 141ms/step - loss: 0.6897 - accuracy: 0.5009 - val_loss: 0.6867 - val_accuracy: 0.4925
Epoch 2/25
181/181 [==============================] - 23s 126ms/step - loss: 0.6830 - accuracy: 0.5043 - val_loss: 0.6820 - val_accuracy: 0.5045
Epoch 3/25
181/181 [==============================] - 20s 112ms/step - loss: 0.6774 - accuracy: 0.5155 - val_loss: 0.6789 - val_accuracy: 0.5176
Epoch 4/25
181/181 [==============================] - 22s 124ms/step - loss: 0.6719 - accuracy: 0.5267 - val_loss: 0.6763 - val_accuracy: 0.5246
Epoch 5/25
181/181 [==============================] - 21s 115ms/step - loss: 0.6662 - accuracy: 0.5384 - val_loss: 0.6740 - val_accuracy: 0.5319
Epoch 6/25
181/181 [==============================] - 21s 114ms/step - loss: 0.6603 - accuracy: 0.5493 - val_loss: 0.6712 - val_accuracy: 0.5371
Epoch 7/25
181/181 [==============================] - 23s 125ms/step - loss: 0.6544 - accuracy: 0.5604 - val_loss: 0.6695 - val_ac

In [5]:

!cd checkpoints/
!mkdir one 
model.save_weights("./checkpoints/one/mycheckpoint")
with open("checkpoints/one/METADATA.txt", 'w') as file:
    file.write("Batch size = " + str(batch_size))
    file.write("Seed = " + str(seed))
    file.write("Sequence length = " + str(sequence_length))
    file.write("Vocab size = " + str(vocab_size))
    file.write("Embedding dimensions = " + str(embedding_dim))
    

mkdir: cannot create directory ‘checkpoints’: File exists


In [20]:
sentence = 'rasekhi left for iran to visit her family the day after trump was elected president her roommate agatha lyczek said ashdkfahsd asdfkasdhfkja asdhfakhsdfj dakjhfaksjdf adhfjasd sahdfkjasd '
guess = model.predict([sentence])[0][0]
if guess > 0.5:
    label = 'liberal'
else:
    label = 'conservative'

print('The sentence ' + "'" + sentence + "'" + ' is ' + label)

The sentence 'rasekhi left for iran to visit her family the day after trump was elected president her roommate agatha lyczek said ashdkfahsd asdfkasdhfkja asdhfakhsdfj dakjhfaksjdf adhfjasd sahdfkjasd ' is conservative
